In [2]:
import qutip as qt 
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

In [3]:
def plot_wigner(
    psi,
    xvec=np.linspace(-5, 5, 101),
    ax=None,
    grid=True,
    yvec=None,
    vmin=-2 / np.pi,
    vmax=+2 / np.pi,
):
    yvec = xvec if yvec is None else yvec
    dx = xvec[1] - xvec[0]
    dy = yvec[1] - yvec[0]
    extent = (
        xvec[0] - dx / 2.0,
        xvec[-1] + dx / 2.0,
        yvec[0] - dy / 2.0,
        yvec[-1] + dy / 2.0,
    )
    W = qt.wigner(psi, xvec,yvec, g=2) 
    
    if ax is None:
        fig = plt.figure()
        ax = plt.gca()
    ax.imshow(
        W,
        origin="lower",
        cmap="seismic",
        vmin=vmin,
        vmax=vmax,
        extent = extent,
        interpolation=None,
    )
    # plt.colorbar()
    if grid:
        ax.grid()
    return fig, ax

def create_gif(states, gif_name, fps = 20, tensored = True):

    if tensored:
        states = [qt.ptrace(state, 1) for state in states]
    # Generate and save frames
    filenames = []
    for i, state in enumerate(states):
        fig, ax = plot_wigner(state)
        filename = f'frame_{i}.png'
        fig.savefig(filename)
        filenames.append(filename)
        plt.close(fig)

    # Create GIF
    with imageio.get_writer(gif_name, mode='I', duration =0.05, fps = fps ) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove frame files
    for filename in filenames:
        os.remove(filename)

In [4]:
# Define System Parameters
N = 100

# Define Operators

# cavity
a = qt.destroy(N)
at = qt.create(N)
Ic = qt.qeye(N)

# qubit
sx = qt.sigmax()
sy = qt.sigmay()
sz = qt.sigmaz()
Iq = qt.qeye(2)

# Define States
g = qt.basis(2, 0)
e = qt.basis(2, 1)

vac = qt.basis(N, 0)

# Define Hamiltonian
def H_dispersive(chi_ge, chi_ef):
    return qt.tensor((-chi_ge * g.proj() + chi_ef * e.proj()),a.dag()*a)

def H_drive(epsilon):
    """Final displacement is epsilon*t"""
    return (np.conjugate(1j*epsilon)*a + a.dag()*1j*epsilon)

def H_rotation(omega,theta):
    """omega: rotation angle
    theta: rotation axis, 0 -> Y, pi/2 -> X
    final rotation angle is omega*t
    """

    return omega/2 * (np.cos(theta) * sy + np.sin(theta) * sx)

# time evolution 
def time_evolution(H, psi, tlist):
    return qt.mesolve(H, psi, tlist, [], [])

# ECD sequence
def ecd_sequence(psi0, epsilon, chi_ge, chi_ef, t_rotation, t_displace, t_wait):
    # 1. pi2 Rotation
    psis1 = time_evolution(qt.tensor(H_rotation(np.pi/2*1/t_rotation[-1], 0),Ic) + H_dispersive(chi_ge,chi_ef), psi0, t_rotation).states
    # 2. First positive displacement
    psis2 = time_evolution(qt.tensor(Iq, H_drive(epsilon/t_displace[-1])) + H_dispersive(chi_ge,chi_ef), psis1[-1], t_displace).states
    # 3. First Wait time
    psis3 = time_evolution(H_dispersive(chi_ge,chi_ef), psis2[-1], t_wait).states
    # 4. First negative displacement
    psis4 = time_evolution(qt.tensor(Iq, H_drive(-epsilon/t_displace[-1]))+ H_dispersive(chi_ge,chi_ef) , psis3[-1], t_displace).states
    # 5. pi Rotation
    psis4 = [psi.tidyup(atol=1e-4) for psi in psis4]
    psis5 = time_evolution(qt.tensor(H_rotation(np.pi*1/t_rotation[-1], 0),Ic) + H_dispersive(chi_ge,chi_ef), psis4[-1], t_rotation).states
    # 6. Second negative displacement
    psis6 = time_evolution(qt.tensor(Iq, H_drive(-epsilon/t_displace[-1])) + H_dispersive(chi_ge,chi_ef), psis5[-1], t_displace).states
    # 7. Second Wait time
    psis7 = time_evolution(H_dispersive(chi_ge,chi_ef), psis6[-1], t_wait).states
    # 8. Second positive displacement
    psis8 = time_evolution(qt.tensor(Iq, H_drive(epsilon/t_displace[-1])) + H_dispersive(chi_ge,chi_ef), psis7[-1], t_displace).states
    return psis1 + psis2 + psis3 + psis4 + psis5 + psis6 + psis7 + psis8

In [13]:
# a dictionary that holds all the parameters for ecd_sequence
ecd_params = {
    'epsilon': 3,
    'chi_ge': 0.250*2*np.pi,
    'chi_ef': 0.125*2*np.pi,
    't_rotation': np.linspace(0, 0.05, 5),
    't_displace': np.linspace(0, 0.05, 5),
    't_wait': np.linspace(0, 0.2, 5)
}
# run the ecd sequence
psi0 = qt.tensor(g, vac)
psis = ecd_sequence(psi0, **ecd_params)

In [16]:
# Specify the folder name
folder_name = "uneven_ecd_gifs"

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
gif_name = f"test_epsilon_{round(ecd_params['epsilon'], 2)}_chi_ge_{round(ecd_params['chi_ge'], 2)}_chi_ef_{round(ecd_params['chi_ef'], 2)}_t_wait_{round(ecd_params['t_wait'][-1], 2)}.gif"

file_path = os.path.join(folder_name, gif_name)

# Save the gif in the specified folder
create_gif(psis, file_path, fps=5)


C:\Users\jonat\AppData\Local\Temp\ipykernel_17860\759877877.py:54: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [20]:
g = qt.basis(2, 0)
e = qt.basis(2, 1)
ge = 1/np.sqrt(2)*(g+e)
ge_vac = qt.tensor(g, vac)
ge_3 = qt.tensor(g, qt.coherent(N, 3))

psis = [ge_vac, ge_3]
psis_displaced = [qt.tensor(Iq,qt.displace(N, 2))*psi for psi in psis]

for psi in psis_displaced:
    print(qt.ptrace(psi, 0))


Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
